In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras import applications
from tensorflow.keras import optimizers

In [18]:
datadir='/home/shashimal/Documents/dataset'
categories = ['dyed-lifted-polyps','dyed-resection-margins','esophagitis',
             'normal-cecum','normal-pylorus','normal-z-line','polyps','ulcerative-colitis']

In [19]:
training_data=[]
def create_training_data():
    for category in categories:
        path = os.path.join(datadir,category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(96,96))
#                 crop_array = new_array[0:96,35:96]
                training_data.append([new_array,class_num])
            except Exception as e:    
                pass
create_training_data() 

In [20]:
x=[]
y=[]

for features,label in training_data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(8000,96,96,3)
x.shape

(8000, 96, 96, 3)

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [22]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train/=255
x_test/=255

In [23]:
y_train = tf.keras.utils.to_categorical(y_train, 8)
y_test = tf.keras.utils.to_categorical(y_test, 8)

In [24]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (96, 96, 3))


In [25]:
for layer in model.layers[:20]:
    layer.trainable = False
x = model.output
x =  tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
predictions = tf.keras.layers.Dense(8, activation="softmax")(x)   
model_final = tf.keras.models.Model(model.input,predictions)
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

    

In [26]:
history=model_final.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 6400 samples, validate on 1600 samples
Epoch 1/20
6400/6400 [==============================] - 567s 89ms/sample - loss: 2.0527 - acc: 0.1991 - val_loss: 1.7468 - val_acc: 0.4137
Epoch 2/20
6400/6400 [==============================] - 558s 87ms/sample - loss: 1.7041 - acc: 0.3742 - val_loss: 1.4542 - val_acc: 0.5638
Epoch 3/20
6400/6400 [==============================] - 578s 90ms/sample - loss: 1.4668 - acc: 0.4675 - val_loss: 1.2694 - val_acc: 0.5794
Epoch 4/20
6400/6400 [==============================] - 551s 86ms/sample - loss: 1.3161 - acc: 0.5067 - val_loss: 1.1311 - val_acc: 0.6075
Epoch 5/20
6400/6400 [==============================] - 534s 83ms/sample - loss: 1.2004 - acc: 0.5417 - val_loss: 1.0440 - val_acc: 0.6219
Epoch 6/20
6400/6400 [==============================] - 555s 87ms/sample - loss: 1.1242 - acc: 0.5559 - val_loss: 0.9820 - val_acc: 0.6338
Epoch 7/20
6400/6400 [==============================] - 603s 94ms/sample - loss: 1.0643 - acc: 0.5784 - val_loss: 0.93

In [ ]:
score = model_final.evaluate(x_test, y_test, verbose=0)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
predictions=model_final.predict(x_test)
predictions[10]

In [ ]:
import numpy as np
np.argmax(predictions[10])

In [ ]:
y_test[10]